## SVD and truncation
We previously mentioned that it is often handy to disregard eigenvalues past a certain value, i.e. truncate the SVD to the first, say, $r$ eigenvalues/eigenvectors. How do we decide how big $r$ needs to be. There are two approaches:
1. Energy-based thresholding: given a target of variance to be capturd (e.g. 90%), this method computes the number of eigenvalues/vectors necessary to hit this target.   
2. Hard thresholding: this is based on theory and needs to know (a) the noise level, the dimension of the matrix (Gavish and Donoho "The Optimal Hard Threshold for Singular Values is $4/\sqrt{3}$"). 

The entire premise of Gavish and Donoho is that you can write your data as:

$$
X = X_{\mathrm{true}} + \gamma X_{\mathrm{noise}}
$$

where $X_{\mathrm{noise}}$ is such that $x_{i,j} \sim \mathcal{N}(0, 1)$. Gavish and Donoho realised that the SVD of $X_{\mathrm{noise}}$ allows them to define a noise floor. Therefore, the optimal threshold to exclude noise it to take all the eigenvalues/vectors above the "noise floor".